In [1]:
import time
import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import lightgbm as lgb
import xgboost as xgb
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [2]:
# 初赛
df_sale_chusai = pd.read_csv('../chusai/data/train_sales_data.csv')
df_search_chusai = pd.read_csv('../chusai/data/train_search_data.csv')
df_user_chusai = pd.read_csv('../chusai/data/train_user_reply_data.csv')
df_eval_chusai = pd.read_csv('../chusai/data/evaluation_public.csv')

sale_chusai = df_sale_chusai.copy()
search_chusai = df_search_chusai.copy()
user_chusai = df_user_chusai.copy()
evaluation_chusai = df_eval_chusai.copy()

data_chusai = pd.merge(sale_chusai, search_chusai, how='left', on=['province', 'adcode', 'model', 'regYear', 'regMonth'])

# 复赛
df_sale = pd.read_csv('./data/train_sales_data.csv')
df_search = pd.read_csv('./data/train_search_data.csv')
df_user = pd.read_csv('./data/train_user_reply_data.csv')
df_eval = pd.read_csv('./data/evaluation_public.csv')

sale = df_sale.copy()
search = df_search.copy()
user = df_user.copy()
evaluation = df_eval.copy()

data = pd.merge(sale, search, how='left', on=['province', 'adcode', 'model', 'regYear', 'regMonth'])
# data = pd.merge(data, user, how='left', on=['model', 'regYear', 'regMonth'])

data_fusai = data[31680:]
evaluation_fusai = evaluation[5280:]

def offline_validation(data, pred='pred', label='label', group='model'):
    data['pred'] = data['pred'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
    data_agg = data.groupby('model').agg({pred:list, label:[list, 'mean']}).reset_index()
    data_agg.columns = ['_'.join(col).strip() for col in data_agg.columns]
    nrmse_score = []
    for raw in data_agg[['{0}_list'.format(pred), '{0}_list'.format(label), '{0}_mean'.format(label)]].values:
        nrmse_score.append(mean_squared_error(raw[0], raw[1]) ** 0.5 / raw[2])
    print(1 - np.mean(nrmse_score))
#     return 1 - np.mean(nrmse_score)

In [3]:
# 去除无用特征
data_fusai.drop(['adcode'], axis=1, inplace=True)
evaluation_fusai.drop(['forecastVolum', 'adcode'], axis=1, inplace=True)

# 给测试集拼接 bodyType
temp = data_fusai[['province', 'model', 'bodyType']].drop_duplicates()
evaluation_fusai = evaluation_fusai.merge(temp, how='left', on=['province', 'model'])

# 编码
le1 = LabelEncoder()
data_fusai['province'] = le1.fit_transform(data_fusai['province'])
evaluation_fusai['province'] = le1.fit_transform(evaluation_fusai['province'])

le2 = LabelEncoder()
data_fusai['model'] = le2.fit_transform(data_fusai['model'])
evaluation_fusai['model'] = le2.fit_transform(evaluation_fusai['model'])

le3 = LabelEncoder()
data_fusai['bodyType'] = le3.fit_transform(data_fusai['bodyType'])
evaluation_fusai['bodyType'] = le3.fit_transform(evaluation_fusai['bodyType'])

# 重新编码月份
data_fusai['regMonth'] = data_fusai.apply(lambda row: row['regMonth'] + (row['regYear']-2016)*12, axis=1).map(int)

# 对salesVolume做log变换
data_fusai['salesVolume'] = np.log1p(data_fusai['salesVolume'])
# 修正popularity
# data_fusai.loc[data_fusai['popularity']>80000, 'popularity'] = 30000
# data_fusai['popularity'] = np.log1p(data_fusai['popularity'])


dataset_fusai = data_fusai[data_fusai['regYear']==2017]

test_fusai = evaluation_fusai.copy()

def distance_to_springfestival(row):
    regYear = row['regYear']
    regMonth = row['regMonth']
     # 16春节：2月，17春节：1月，18春节：2月
    if regYear==2016:   
        if regMonth!=12:
            return regMonth-2
        else:
            return -1
    elif regYear==2017:
        return regMonth-13
    else:
        return regMonth-26
    
def tongji(row, mt_num=1, which='sale', method='mean'):
    mt = []
    for i in range(mt_num):
        mt.append(row['mt' + str(i+1) + '_' + which])
#     mt.append(row['mt2_' + which])
#     mt.append(row['mt3_' + which])
#     mt.append(row['mt4_' + which])
#     mt.append(row['mt5_' + which])
#     mt.append(row['mt6_' + which])
    ls = mt[:mt_num]
    
    if method=='mean':
        return np.mean(ls)
    elif method=='max':
        return np.max(ls)
    elif method=='min':
        return np.min(ls)
    elif method=='median':
        return np.median(ls)
    elif method=='var':
        return np.var(ls)
    elif method=='std':
        return np.median(ls)
    else:
        return 0

C:\ProgramFiles\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pan

In [4]:
def bulid_model(X_train, y_train, X_valid=None, y_valid=None, cate_feat=None):
    lgb_train = lgb.Dataset(X_train, y_train)
#     if X_valid is not None:
#     lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
    params = {
        'boosting_type': 'gbdt',
        'objective': 'mse',
        'metric': 'mse',       # 'l2', 'binary_logloss', 'auc'
        'learning_rate': 0.05,
        'boost_from_average': False,
    #     'min_data_in_leaf':50,
    #     'num_leaves': 30,
        'max_depth': -1,
        'subsample': 0.8,
    #     'subsample_freq': 5,
        'colsample_bytree': 0.8,
    #     'bagging_freq': 5,
        'seed': 666,
        'nthread': -1,
        'verbose': 1,
    }
    lgb_model = lgb.train(params, lgb_train, num_boost_round=3000, categorical_feature=cate_feat)
    return lgb_model

In [5]:
def extract_feature(data, history, gap=1):
    
    dataset = data.copy()
    hist = history.copy()
    dataset.reset_index(drop=True, inplace=True)
    hist.reset_index(drop=True, inplace=True)
    
    print('Before data shape: ', dataset.shape)
    print('-' * 30)
    
    '''历史特征'''
    # sale
    last_mt_sales, last_y_pops = [], []
    prev_mt1_sales, prev_mt2_sales, prev_mt3_sales, prev_mt4_sales, prev_mt5_sales, prev_mt6_sales = [], [], [], [], [], []
    prev_mt7_sales, prev_mt8_sales, prev_mt9_sales, prev_mt10_sales, prev_mt11_sales, prev_mt12_sales = [], [], [], [], [], []
    
    last_mt_pops, last_y_pops = [], []
    prev_mt1_pops, prev_mt2_pops, prev_mt3_pops, prev_mt4_pops, prev_mt5_pops, prev_mt6_pops = [], [], [], [], [], []
    prev_mt7_pops, prev_mt8_pops, prev_mt9_pops, prev_mt10_pops, prev_mt11_pops, prev_mt12_pops = [], [], [], [], [], []
    
    for row in dataset.itertuples():
#         row = dataset[i:i+1][['province', 'model', 'regYear', 'regMonth']]
#         values = row.values.tolist()[0]
        province, model, regYear, regMonth = row.province, row.model, row.regYear, row.regMonth
        # 去年同月
        last_mt_sale = hist[(hist['province']==province) & (hist['model']==model) & (hist['regMonth']==regMonth-12)]['salesVolume'].values[0]
        last_mt_sales.append(last_mt_sale)
        last_mt_pop = hist[(hist['province']==province) & (hist['model']==model) & (hist['regMonth']==regMonth-12)]['popularity'].values[0]
        last_mt_pops.append(last_mt_pop)
        # 前几个月sale
        prev_sale1 = hist[(hist['province']==province) & (hist['model']==model) & (hist['regMonth']==regMonth-1)]['salesVolume'].values[0]
        prev_mt1_sales.append(prev_sale1)
        prev_sale2 = hist[(hist['province']==province) & (hist['model']==model) & (hist['regMonth']==regMonth-2)]['salesVolume'].values[0]
        prev_mt2_sales.append(prev_sale2)
        prev_sale3 = hist[(hist['province']==province) & (hist['model']==model) & (hist['regMonth']==regMonth-3)]['salesVolume'].values[0]
        prev_mt3_sales.append(prev_sale3)
        prev_sale4 = hist[(hist['province']==province) & (hist['model']==model) & (hist['regMonth']==regMonth-4)]['salesVolume'].values[0]
        prev_mt4_sales.append(prev_sale4)
        prev_sale5 = hist[(hist['province']==province) & (hist['model']==model) & (hist['regMonth']==regMonth-5)]['salesVolume'].values[0]
        prev_mt5_sales.append(prev_sale5)
        prev_sale6 = hist[(hist['province']==province) & (hist['model']==model) & (hist['regMonth']==regMonth-6)]['salesVolume'].values[0]
        prev_mt6_sales.append(prev_sale6)
        prev_sale7 = hist[(hist['province']==province) & (hist['model']==model) & (hist['regMonth']==regMonth-7)]['salesVolume'].values[0]
        prev_mt7_sales.append(prev_sale7)
        prev_sale8 = hist[(hist['province']==province) & (hist['model']==model) & (hist['regMonth']==regMonth-8)]['salesVolume'].values[0]
        prev_mt8_sales.append(prev_sale8)
        prev_sale9 = hist[(hist['province']==province) & (hist['model']==model) & (hist['regMonth']==regMonth-9)]['salesVolume'].values[0]
        prev_mt9_sales.append(prev_sale9)
        prev_sale10 = hist[(hist['province']==province) & (hist['model']==model) & (hist['regMonth']==regMonth-10)]['salesVolume'].values[0]
        prev_mt10_sales.append(prev_sale10)
        prev_sale11 = hist[(hist['province']==province) & (hist['model']==model) & (hist['regMonth']==regMonth-11)]['salesVolume'].values[0]
        prev_mt11_sales.append(prev_sale11)
        # 前几个月pop
        prev_pop1 = hist[(hist['province']==province) & (hist['model']==model) & (hist['regMonth']==regMonth-1)]['popularity'].values[0]
        prev_mt1_pops.append(prev_pop1)
        prev_pop2 = hist[(hist['province']==province) & (hist['model']==model) & (hist['regMonth']==regMonth-2)]['popularity'].values[0]
        prev_mt2_pops.append(prev_pop2)
        prev_pop3 = hist[(hist['province']==province) & (hist['model']==model) & (hist['regMonth']==regMonth-3)]['popularity'].values[0]
        prev_mt3_pops.append(prev_pop3)
        prev_pop4 = hist[(hist['province']==province) & (hist['model']==model) & (hist['regMonth']==regMonth-4)]['popularity'].values[0]
        prev_mt4_pops.append(prev_pop4)
        prev_pop5 = hist[(hist['province']==province) & (hist['model']==model) & (hist['regMonth']==regMonth-5)]['popularity'].values[0]
        prev_mt5_pops.append(prev_pop5)
        prev_pop6 = hist[(hist['province']==province) & (hist['model']==model) & (hist['regMonth']==regMonth-6)]['popularity'].values[0]
        prev_mt6_pops.append(prev_pop6)
        prev_pop7 = hist[(hist['province']==province) & (hist['model']==model) & (hist['regMonth']==regMonth-7)]['popularity'].values[0]
        prev_mt7_pops.append(prev_pop7)
        prev_pop8 = hist[(hist['province']==province) & (hist['model']==model) & (hist['regMonth']==regMonth-8)]['popularity'].values[0]
        prev_mt8_pops.append(prev_pop8)
        prev_pop9 = hist[(hist['province']==province) & (hist['model']==model) & (hist['regMonth']==regMonth-9)]['popularity'].values[0]
        prev_mt9_pops.append(prev_pop9)
        prev_pop10 = hist[(hist['province']==province) & (hist['model']==model) & (hist['regMonth']==regMonth-10)]['popularity'].values[0]
        prev_mt10_pops.append(prev_pop10)
        prev_pop11 = hist[(hist['province']==province) & (hist['model']==model) & (hist['regMonth']==regMonth-11)]['popularity'].values[0]
        prev_mt11_pops.append(prev_pop11)
        
    temp = pd.DataFrame()
    temp['last_sale'] = last_mt_sales
    temp['mt1_sale'] = prev_mt1_sales
    temp['mt2_sale'] = prev_mt2_sales
    temp['mt3_sale'] = prev_mt3_sales
    temp['mt4_sale'] = prev_mt4_sales
    temp['mt5_sale'] = prev_mt5_sales
    temp['mt6_sale'] = prev_mt6_sales
    temp['mt7_sale'] = prev_mt7_sales
    temp['mt8_sale'] = prev_mt8_sales
    temp['mt9_sale'] = prev_mt9_sales
    temp['mt10_sale'] = prev_mt10_sales
    temp['mt11_sale'] = prev_mt11_sales
    temp['last_pop'] = last_mt_pops
    temp['mt1_pop'] = prev_mt1_pops
    temp['mt2_pop'] = prev_mt2_pops
    temp['mt3_pop'] = prev_mt3_pops
    temp['mt4_pop'] = prev_mt4_pops
    temp['mt5_pop'] = prev_mt5_pops
    temp['mt6_pop'] = prev_mt6_pops
    temp['mt7_pop'] = prev_mt7_pops
    temp['mt8_pop'] = prev_mt8_pops
    temp['mt9_pop'] = prev_mt9_pops
    temp['mt10_pop'] = prev_mt10_pops
    temp['mt11_pop'] = prev_mt11_pops
    dataset = pd.concat([dataset, temp], axis=1)

    # 直接差分
    dataset['mt1_2_sale'] = dataset['mt1_sale'] - dataset['mt2_sale']
    dataset['mt2_3_sale'] = dataset['mt2_sale'] - dataset['mt3_sale']
    dataset['mt3_4_sale'] = dataset['mt3_sale'] - dataset['mt4_sale']
    dataset['mt4_5_sale'] = dataset['mt4_sale'] - dataset['mt5_sale']
    dataset['mt5_6_sale'] = dataset['mt5_sale'] - dataset['mt6_sale']
    dataset['mt6_7_sale'] = dataset['mt6_sale'] - dataset['mt7_sale']
    dataset['mt7_8_sale'] = dataset['mt7_sale'] - dataset['mt8_sale']
    dataset['mt8_9_sale'] = dataset['mt8_sale'] - dataset['mt9_sale']
    dataset['mt9_10_sale'] = dataset['mt9_sale'] - dataset['mt10_sale']
    dataset['mt10_11_sale'] = dataset['mt10_sale'] - dataset['mt11_sale']
    dataset['mt11_12_sale'] = dataset['mt11_sale'] - dataset['last_sale']
    dataset['mt_tongbi_sale'] = dataset['mt1_sale'] - dataset['last_sale']
    dataset['mt1_2_sale_rate'] = dataset['mt1_2_sale'] / dataset['mt1_sale']
    dataset['mt2_3_sale_rate'] = dataset['mt2_3_sale'] / dataset['mt2_sale']
    dataset['mt3_4_sale_rate'] = dataset['mt3_4_sale'] / dataset['mt3_sale']
    dataset['mt4_5_sale_rate'] = dataset['mt4_5_sale'] / dataset['mt4_sale']
    dataset['mt5_6_sale_rate'] = dataset['mt5_6_sale'] / dataset['mt5_sale']
    dataset['mt6_7_sale_rate'] = dataset['mt6_7_sale'] / dataset['mt6_sale']
    dataset['mt7_8_sale_rate'] = dataset['mt7_8_sale'] / dataset['mt7_sale']
    dataset['mt8_9_sale_rate'] = dataset['mt8_9_sale'] / dataset['mt8_sale']
    dataset['mt9_10_sale_rate'] = dataset['mt9_10_sale'] / dataset['mt9_sale']
    dataset['mt10_11_sale_rate'] = dataset['mt10_11_sale'] / dataset['mt10_sale']
    dataset['mt11_12_sale_rate'] = dataset['mt11_12_sale'] / dataset['mt11_sale']
    dataset['mt_tongbi_sale_rate'] = dataset['mt1_sale'] / dataset['last_sale']
    
    dataset['mt1_2_pop'] = dataset['mt1_pop'] - dataset['mt2_pop']
    dataset['mt2_3_pop'] = dataset['mt2_pop'] - dataset['mt3_pop']
    dataset['mt3_4_pop'] = dataset['mt3_pop'] - dataset['mt4_pop']
    dataset['mt4_5_pop'] = dataset['mt4_pop'] - dataset['mt5_pop']
    dataset['mt5_6_pop'] = dataset['mt5_pop'] - dataset['mt6_pop']
    dataset['mt6_7_pop'] = dataset['mt6_pop'] - dataset['mt7_pop']
    dataset['mt7_8_pop'] = dataset['mt7_pop'] - dataset['mt8_pop']
    dataset['mt8_9_pop'] = dataset['mt8_pop'] - dataset['mt9_pop']
    dataset['mt9_10_pop'] = dataset['mt9_pop'] - dataset['mt10_pop']
    dataset['mt10_11_pop'] = dataset['mt10_pop'] - dataset['mt11_pop']
    dataset['mt11_12_pop'] = dataset['mt11_pop'] - dataset['last_pop']
    dataset['mt_tongbi_pop'] = dataset['mt1_pop'] - dataset['last_pop']
    dataset['mt1_2_pop_rate'] = dataset['mt1_2_pop'] / dataset['mt1_pop']
    dataset['mt2_3_pop_rate'] = dataset['mt2_3_pop'] / dataset['mt2_pop']
    dataset['mt3_4_pop_rate'] = dataset['mt3_4_pop'] / dataset['mt3_pop']
    dataset['mt4_5_pop_rate'] = dataset['mt4_5_pop'] / dataset['mt4_pop']
    dataset['mt5_6_pop_rate'] = dataset['mt5_6_pop'] / dataset['mt5_pop']
    dataset['mt6_7_pop_rate'] = dataset['mt6_7_pop'] / dataset['mt6_pop']
    dataset['mt7_8_pop_rate'] = dataset['mt7_8_pop'] / dataset['mt7_pop']
    dataset['mt8_9_pop_rate'] = dataset['mt8_9_pop'] / dataset['mt8_pop']
    dataset['mt9_10_pop_rate'] = dataset['mt9_10_pop'] / dataset['mt9_pop']
    dataset['mt10_11_pop_rate'] = dataset['mt10_11_pop'] / dataset['mt10_pop']
    dataset['mt11_12_pop_rate'] = dataset['mt11_12_pop'] / dataset['last_pop']
    dataset['mt_tongbi_pop_rate'] = dataset['mt1_pop'] / dataset['last_pop']
    
    dataset['distance_to_springfestival'] = dataset.apply(lambda row: distance_to_springfestival(row), axis=1).map(int)

    # 6个月 sale
    dataset['6mt_mean_sale'] = dataset.apply(lambda row: tongji(row, 6, 'sale', 'mean'), axis=1)
    dataset['6mt_max_sale'] = dataset.apply(lambda row: tongji(row, 6, 'sale', 'max'), axis=1)
    dataset['6mt_min_sale'] = dataset.apply(lambda row: tongji(row, 6, 'sale', 'min'), axis=1)
    dataset['6mt_median_sale'] = dataset.apply(lambda row: tongji(row, 6, 'sale', 'median'), axis=1)
    dataset['6mt_var_sale'] = dataset.apply(lambda row: tongji(row, 6, 'sale', 'var'), axis=1)
#     dataset['6mt_std_sale'] = dataset.apply(lambda row: tongji(row, 6, 'sale', 'std'), axis=1)
#     dataset['6mt_sum_sale'] = dataset.apply(lambda row: tongji(row, 6, 'sale', 'sum'), axis=1)
    
    dataset['6mt_mean_pop'] = dataset.apply(lambda row: tongji(row, 6, 'pop', 'mean'), axis=1)
    dataset['6mt_max_pop'] = dataset.apply(lambda row: tongji(row, 6, 'pop', 'max'), axis=1)
    dataset['6mt_min_pop'] = dataset.apply(lambda row: tongji(row, 6, 'pop', 'min'), axis=1)
    dataset['6mt_median_pop'] = dataset.apply(lambda row: tongji(row, 6, 'pop', 'median'), axis=1)
    dataset['6mt_var_pop'] = dataset.apply(lambda row: tongji(row, 6, 'pop', 'var'), axis=1)
#     dataset['6mt_std_pop'] = dataset.apply(lambda row: tongji(row, 6, 'pop', 'std'), axis=1)
#     dataset['6mt_sum_pop'] = dataset.apply(lambda row: tongji(row, 6, 'pop', 'sum'), axis=1)
    # 4个月
    dataset['4mt_mean_sale'] = dataset.apply(lambda row: tongji(row, 4, 'sale', 'mean'), axis=1)
    dataset['4mt_max_sale'] = dataset.apply(lambda row: tongji(row, 4, 'sale', 'max'), axis=1)
    dataset['4mt_min_sale'] = dataset.apply(lambda row: tongji(row, 4, 'sale', 'min'), axis=1)
    dataset['4mt_median_sale'] = dataset.apply(lambda row: tongji(row, 4, 'sale', 'median'), axis=1)
    dataset['4mt_var_sale'] = dataset.apply(lambda row: tongji(row, 4, 'sale', 'var'), axis=1)
#     dataset['4mt_std_sale'] = dataset.apply(lambda row: tongji(row, 4, 'sale', 'std'), axis=1)
#     dataset['4mt_sum_sale'] = dataset.apply(lambda row: tongji(row, 4, 'sale', 'sum'), axis=1)
    
    dataset['4mt_mean_pop'] = dataset.apply(lambda row: tongji(row, 4, 'pop', 'mean'), axis=1)
    dataset['4mt_max_pop'] = dataset.apply(lambda row: tongji(row, 4, 'pop', 'max'), axis=1)
    dataset['4mt_min_pop'] = dataset.apply(lambda row: tongji(row, 4, 'pop', 'min'), axis=1)
    dataset['4mt_median_pop'] = dataset.apply(lambda row: tongji(row, 4, 'pop', 'median'), axis=1)
    dataset['4mt_var_pop'] = dataset.apply(lambda row: tongji(row, 4, 'pop', 'var'), axis=1)
#     dataset['4mt_std_pop'] = dataset.apply(lambda row: tongji(row, 4, 'pop', 'std'), axis=1)
#     dataset['4mt_sum_pop'] = dataset.apply(lambda row: tongji(row, 4, 'pop', 'sum'), axis=1)
    # 3个月 
    dataset['3mt_mean_sale'] = dataset.apply(lambda row: tongji(row, 3, 'sale', 'mean'), axis=1)
    dataset['3mt_max_sale'] = dataset.apply(lambda row: tongji(row, 3, 'sale', 'max'), axis=1)
    dataset['3mt_min_sale'] = dataset.apply(lambda row: tongji(row, 3, 'sale', 'min'), axis=1)
    dataset['3mt_median_sale'] = dataset.apply(lambda row: tongji(row, 3, 'sale', 'median'), axis=1)
    dataset['3mt_var_sale'] = dataset.apply(lambda row: tongji(row, 3, 'sale', 'var'), axis=1)
#     dataset['3mt_std_sale'] = dataset.apply(lambda row: tongji(row, 3, 'sale', 'std'), axis=1)
#     dataset['3mt_sum_sale'] = dataset.apply(lambda row: tongji(row, 3, 'sale', 'sum'), axis=1)
    
    dataset['3mt_mean_pop'] = dataset.apply(lambda row: tongji(row, 3, 'pop', 'mean'), axis=1)
    dataset['3mt_max_pop'] = dataset.apply(lambda row: tongji(row, 3, 'pop', 'max'), axis=1)
    dataset['3mt_min_pop'] = dataset.apply(lambda row: tongji(row, 3, 'pop', 'min'), axis=1)
    dataset['3mt_median_pop'] = dataset.apply(lambda row: tongji(row, 3, 'pop', 'median'), axis=1)
    dataset['3mt_var_pop'] = dataset.apply(lambda row: tongji(row, 3, 'pop', 'var'), axis=1)
#     dataset['3mt_std_pop'] = dataset.apply(lambda row: tongji(row, 3, 'pop', 'std'), axis=1)
#     dataset['3mt_sum_pop'] = dataset.apply(lambda row: tongji(row, 3, 'pop', 'sum'), axis=1)
    # 2个月
    dataset['2mt_mean_sale'] = dataset.apply(lambda row: tongji(row, 2, 'sale', 'mean'), axis=1)
    dataset['2mt_max_sale'] = dataset.apply(lambda row: tongji(row, 2, 'sale', 'max'), axis=1)
    dataset['2mt_min_sale'] = dataset.apply(lambda row: tongji(row, 2, 'sale', 'min'), axis=1)
    dataset['2mt_median_sale'] = dataset.apply(lambda row: tongji(row, 2, 'sale', 'median'), axis=1)
    dataset['2mt_var_sale'] = dataset.apply(lambda row: tongji(row, 2, 'sale', 'var'), axis=1)
#     dataset['2mt_std_sale'] = dataset.apply(lambda row: tongji(row, 2, 'sale', 'std'), axis=1)
#     dataset['2mt_sum_sale'] = dataset.apply(lambda row: tongji(row, 2, 'sale', 'sum'), axis=1)
    
    dataset['2mt_mean_pop'] = dataset.apply(lambda row: tongji(row, 2, 'pop', 'mean'), axis=1)
    dataset['2mt_max_pop'] = dataset.apply(lambda row: tongji(row, 2, 'pop', 'max'), axis=1)
    dataset['2mt_min_pop'] = dataset.apply(lambda row: tongji(row, 2, 'pop', 'min'), axis=1)
    dataset['2mt_median_pop'] = dataset.apply(lambda row: tongji(row, 2, 'pop', 'median'), axis=1)
    dataset['2mt_var_pop'] = dataset.apply(lambda row: tongji(row, 2, 'pop', 'var'), axis=1)
#     dataset['2mt_std_pop'] = dataset.apply(lambda row: tongji(row, 2, 'pop', 'std'), axis=1)
#     dataset['2mt_sum_pop'] = dataset.apply(lambda row: tongji(row, 2, 'pop', 'sum'), axis=1)
    
    print('After data shape: ', dataset.shape)
    print('='*30)
    
    return dataset

In [6]:
start_time=time.time()
train_feature = extract_feature(dataset_fusai, data_fusai)
end_time=time.time()
print('Time cost(s): ', end_time-start_time)

Before data shape:  (5808, 7)
------------------------------
After data shape:  (5808, 120)
Time cost(s):  271.5604248046875


In [7]:
cate_feat = ['province', 'model', 'bodyType', 'regYear', 'regMonth'] #, 'bodyType'

X_train = train_feature[train_feature['regMonth'].between(13, 24)]
y_train = X_train[['salesVolume']]

X_train.drop(['salesVolume', 'popularity'], axis=1, inplace=True)

lgb_model = bulid_model(X_train, y_train, cate_feat=cate_feat)

C:\ProgramFiles\Anaconda3\lib\site-packages\lightgbm\basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['bodyType', 'model', 'province', 'regMonth', 'regYear']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


### Month1

In [8]:
test_fusai['regMonth'] = test_fusai.apply(lambda row: row['regMonth'] + (row['regYear']-2016)*12, axis=1).map(int)

In [9]:
start_time=time.time()
test_feature_mt1 = extract_feature(test_fusai[test_fusai['regMonth']==25], data_fusai)
end_time=time.time()
print('Time cost(s): ', end_time-start_time)

Before data shape:  (484, 6)
------------------------------
After data shape:  (484, 119)
Time cost(s):  23.95292901992798


In [10]:
mytest1 = test_feature_mt1.copy()
X_test1 = mytest1.drop('id', axis=1)
y_test1 = mytest1[['id']]

pred_log = lgb_model.predict(X_test1, num_iteration=lgb_model.best_iteration)
test_feature_mt1['salesVolume'] = pred_log
test_feature_mt1['salesVolume'] = test_feature_mt1['salesVolume'].apply(lambda x: x if x>0 else 0)
# test_feature_mt1['salesVolume'] = np.round(test_feature_mt1['salesVolume']).astype(int)

print('Average forecastVolum:\t', np.expm1(test_feature_mt1['salesVolume']).mean())
print('Min forecastVolum:\t', np.expm1(test_feature_mt1['salesVolume']).min())
print('Max forecastVolum:\t', np.expm1(test_feature_mt1['salesVolume']).max())
print('Total forecastVolum:\t', np.expm1(test_feature_mt1['salesVolume']).sum())

sub_test_feature_mt1 = test_feature_mt1[['province', 'model', 'bodyType', 'regYear', 'regMonth', 'salesVolume']]
data_fusai = pd.concat([data_fusai, sub_test_feature_mt1], axis=0)

Average forecastVolum:	 237.0024670938374
Min forecastVolum:	 1.6768337632384211
Max forecastVolum:	 2517.3386019047944
Total forecastVolum:	 114709.19407341728


C:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


### Month2

In [11]:
start_time=time.time()
test_feature_mt2 = extract_feature(test_fusai[test_fusai['regMonth']==26], data_fusai)
end_time=time.time()
print('Time cost(s): ', end_time-start_time)

Before data shape:  (484, 6)
------------------------------
After data shape:  (484, 119)
Time cost(s):  23.926126956939697


In [12]:
mytest2 = test_feature_mt2.copy()
X_test2 = mytest2.drop('id', axis=1)
y_test2 = mytest2[['id']]

pred_log = lgb_model.predict(X_test2, num_iteration=lgb_model.best_iteration)
test_feature_mt2['salesVolume'] = pred_log
test_feature_mt2['salesVolume'] = test_feature_mt2['salesVolume'].apply(lambda x: x if x>0 else 0)
# test_feature_mt2['salesVolume'] = np.round(test_feature_mt2['salesVolume']).astype(int)

print('Average forecastVolum:\t', np.expm1(test_feature_mt2['salesVolume']).mean())
print('Min forecastVolum:\t', np.expm1(test_feature_mt2['salesVolume']).min())
print('Max forecastVolum:\t', np.expm1(test_feature_mt2['salesVolume']).max())
print('Total forecastVolum:\t', np.expm1(test_feature_mt2['salesVolume']).sum())

sub_test_feature_mt2 = test_feature_mt2[['province', 'model', 'bodyType', 'regYear', 'regMonth', 'salesVolume']]
data_fusai = pd.concat([data_fusai, sub_test_feature_mt2], axis=0)

Average forecastVolum:	 172.14275082230662
Min forecastVolum:	 1.5811361702900926
Max forecastVolum:	 1608.0346526752553
Total forecastVolum:	 83317.09139799635


C:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


### Month3

In [13]:
start_time=time.time()
test_feature_mt3 = extract_feature(test_fusai[test_fusai['regMonth']==27], data_fusai)
end_time=time.time()
print('Time cost(s): ', end_time-start_time)

Before data shape:  (484, 6)
------------------------------
After data shape:  (484, 119)
Time cost(s):  23.93682050704956


In [14]:
mytest3 = test_feature_mt3.copy()
X_test3 = mytest3.drop('id', axis=1)
y_test3 = mytest3[['id']]

pred_log = lgb_model.predict(X_test3, num_iteration=lgb_model.best_iteration)
test_feature_mt3['salesVolume'] = pred_log
test_feature_mt3['salesVolume'] = test_feature_mt3['salesVolume'].apply(lambda x: x if x>0 else 0)
# test_feature_mt3['salesVolume'] = np.round(test_feature_mt3['salesVolume']).astype(int)

print('Average forecastVolum:\t', np.expm1(test_feature_mt3['salesVolume']).mean())
print('Min forecastVolum:\t', np.expm1(test_feature_mt3['salesVolume']).min())
print('Max forecastVolum:\t', np.expm1(test_feature_mt3['salesVolume']).max())
print('Total forecastVolum:\t', np.expm1(test_feature_mt3['salesVolume']).sum())

sub_test_feature_mt3 = test_feature_mt3[['province', 'model', 'bodyType', 'regYear', 'regMonth', 'salesVolume']]
data_fusai = pd.concat([data_fusai, sub_test_feature_mt3], axis=0)

Average forecastVolum:	 144.42664769803858
Min forecastVolum:	 1.2933912759284634
Max forecastVolum:	 1290.1166097232779
Total forecastVolum:	 69902.49748585065


C:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


### Month4

In [15]:
start_time=time.time()
test_feature_mt4 = extract_feature(test_fusai[test_fusai['regMonth']==28], data_fusai)
end_time=time.time()
print('Time cost(s): ', end_time-start_time)

Before data shape:  (484, 6)
------------------------------
After data shape:  (484, 119)
Time cost(s):  24.34208059310913


In [16]:
mytest4 = test_feature_mt4.copy()
X_test4 = mytest4.drop('id', axis=1)
y_test4 = mytest4[['id']]

pred_log = lgb_model.predict(X_test4, num_iteration=lgb_model.best_iteration)
test_feature_mt4['salesVolume'] = pred_log
test_feature_mt4['salesVolume'] = test_feature_mt4['salesVolume'].apply(lambda x: x if x>0 else 0)
# test_feature_mt4['salesVolume'] = np.round(test_feature_mt4['salesVolume']).astype(int)

print('Average forecastVolum:\t', np.expm1(test_feature_mt4['salesVolume']).mean())
print('Min forecastVolum:\t', np.expm1(test_feature_mt4['salesVolume']).min())
print('Max forecastVolum:\t', np.expm1(test_feature_mt4['salesVolume']).max())
print('Total forecastVolum:\t', np.expm1(test_feature_mt4['salesVolume']).sum())

Average forecastVolum:	 167.35664753832012
Min forecastVolum:	 1.8885755703214637
Max forecastVolum:	 1670.4455560616393
Total forecastVolum:	 81000.61740854697


### Submission

In [17]:
lgb_pred = pd.concat([y_test1, y_test2, y_test3, y_test4])
pred = pd.concat([test_feature_mt1['salesVolume'], test_feature_mt2['salesVolume'], test_feature_mt3['salesVolume'], test_feature_mt4['salesVolume']])

lgb_pred['forecastVolum'] = pred
lgb_pred['forecastVolum'] = np.expm1(lgb_pred['forecastVolum'])#expm1
lgb_pred['forecastVolum'] = np.round(lgb_pred['forecastVolum']).astype(int)
lgb_pred[['id', 'forecastVolum']].to_csv('./submit/lgb_ccy.csv', encoding='utf-8', index=None)

In [18]:
lgb_pred.forecastVolum.mean()

180.23347107438016